In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime

# Tickers and how many expirations to collect per ticker
tickers = ["SPY", "AAPL", "QQQ", "TSLA", "MSFT"]
max_expirations = 4  # fetch next 4 expiration dates

all_options = []

for ticker_symbol in tickers:
    ticker = yf.Ticker(ticker_symbol)
    expirations = ticker.options[:max_expirations]
    
    for exp_date in expirations:
        try:
            opt_chain = ticker.option_chain(exp_date).calls
            spot_price = ticker.history(period="1d")["Close"].iloc[-1]
            expiry_date = pd.to_datetime(exp_date)
            days_to_expiry = (expiry_date - pd.Timestamp.today()).days

            opt_chain["ticker"] = ticker_symbol
            opt_chain["expiry"] = expiry_date
            opt_chain["days_to_expiry"] = days_to_expiry
            opt_chain["spot_price"] = spot_price
            opt_chain["moneyness"] = opt_chain["strike"] / spot_price
            opt_chain["date_collected"] = pd.Timestamp.today()

            all_options.append(opt_chain)
        except Exception as e:
            print(f"Error with {ticker_symbol} on {exp_date}: {e}")

# Combine into one DataFrame
df_all = pd.concat(all_options).reset_index(drop=True)

# Optional cleanup
df_all = df_all[df_all['impliedVolatility'] < 5]
df_all = df_all.dropna()

# Save to CSV
df_all.to_csv("../data/raw/multi_ticker_options.csv", index=False)
print("✅ Saved expanded options dataset with shape:", df_all.shape)

✅ Saved expanded options dataset with shape: (1869, 20)
